# Byte Pair Encoding (BPE) Tokenizer From Scratch
包含可选（奖励）代码，解释并展示BPE分词器的工作原理。

- 这是一个独立的notebook，从零开始实现流行的字节对编码（BPE）分词算法，该算法用于GPT-2到GPT-4、Llama 3等模型，专为教育目的设计
- 有关分词目的的更多详细信息，请参考[第2章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这里的代码是解释BPE算法的额外材料
- OpenAI为训练原始GPT模型实现的原始BPE分词器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到
- BPE算法最初于1994年由Philip Gage在"[数据压缩的新算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"中描述
- 现今大多数项目，包括Llama 3，都使用OpenAI的开源[tiktoken库](https://github.com/openai/tiktoken)，因为其计算性能优秀；它允许加载预训练的GPT-2和GPT-4分词器，例如（Llama 3模型也使用GPT-4分词器进行训练）
- 上述实现与我在此notebook中的实现之间的区别在于，我的实现还包含了用于训练分词器的函数（用于教育目的）
- 还有一个叫做[minBPE](https://github.com/karpathy/minbpe)的实现支持训练，可能性能更好（我这里的实现专注于教育目的）；与`minBPE`相比，我的实现还允许加载原始OpenAI分词器词汇表和BPE"合并"（此外，Hugging Face分词器也能够训练和加载各种分词器；请参阅读者在尼泊尔语上训练BPE分词器的[GitHub讨论](https://github.com/rasbt/LLMs-from-scratch/discussions/485)了解更多信息）

&nbsp;
# 1. 字节对编码（BPE）背后的主要思想

- BPE的主要思想是将文本转换为整数表示（token ID），用于LLM训练（参见[第2章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">

&nbsp;
## 1.1 位和字节

- 在深入BPE算法之前，让我们先介绍字节的概念
- 考虑将文本转换为字节数组（毕竟BPE代表"字节"对编码）：

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


- 当我们对`bytearray`对象调用`list()`时，每个字节被视为单独的元素，结果是与字节值对应的整数列表：

In [2]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- 这是将文本转换为LLM的嵌入层所需的token ID表示的有效方法
- 然而，这种方法的缺点是为每个字符创建一个ID（对于短文本来说这是很多ID！）
- 即，这意味着对于17个字符的输入文本，我们必须使用17个token ID作为LLM的输入：

In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果您之前使用过LLM，您可能知道BPE分词器有一个词汇表，其中我们为整个单词或子词而不是每个字符都有token ID
- 例如，GPT-2分词器将相同的文本（"This is some text"）仅分词为4个而不是17个token：`1212, 318, 617, 2420`
- 您可以使用交互式[tiktoken应用](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken库](https://github.com/openai/tiktoken)再次检查：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# 输出 [1212, 318, 617, 2420]
```

- 由于一个字节由8位组成，单个字节可以表示2<sup>8</sup> = 256个可能的值，范围从0到255
- 您可以通过执行代码`bytearray(range(0, 257))`来确认这一点，这将警告您`ValueError: byte must be in range(0, 256)`）
- BPE分词器通常使用这256个值作为其前256个单字符token；可以通过运行以下代码直观地检查这一点：

```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
输出：
0: !
1: "
2: #
...
255: �  # <---- 到这里都是单字符token
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上面要注意的是，条目256和257不是单字符值而是双字符值（一个空格+一个字母），这是原始GPT-2 BPE分词器的一个小缺点（这在GPT-4分词器中已经得到改进）

&nbsp;
## 1.2 构建词汇表

- BPE分词算法的目标是构建一个常见子词的词汇表，如`298: ent`（可以在*entangle, entertain, enter, entrance, entity, ...*等词中找到），甚至是完整的单词，如 

```
318: is
617: some
1212: This
2420: text
```

- BPE算法最初于1994年由Philip Gage在"[数据压缩的新算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"中描述
- 在进入实际代码实现之前，今天用于LLM分词器的形式可以总结为以下部分中描述的内容。

&nbsp;
## 1.3 BPE算法概述

**1. 识别频繁对**
- 在每次迭代中，扫描文本以找到最常出现的字节（或字符）对

**2. 替换和记录**

- 用一个新的占位符ID替换该对（一个尚未使用的ID，例如，如果我们从0...255开始，第一个占位符将是256）
- 在查找表中记录此映射
- 查找表的大小是一个超参数，也称为"词汇表大小"（对于GPT-2，这是
50,257）

**3. 重复直到无收益**

- 持续重复步骤1和2，不断合并最频繁的对
- 当无法进一步压缩时停止（例如，没有对出现超过一次）

**解压缩（解码）**

- 要恢复原始文本，通过使用查找表将每个ID替换为其对应的对来逆转过程


&nbsp;
## 1.4 BPE算法示例

### 1.4.1 编码部分的具体示例（第1.3节中的步骤1和2）

- 假设我们有文本（训练数据集）`the cat in the hat`，我们想从中构建BPE分词器的词汇表

**迭代1**

1. 识别频繁对
  - 在此文本中，"th"出现两次（在开头和第二个"e"之前）

2. 替换和记录
  - 用一个尚未使用的新token ID替换"th"，例如256
  - 新文本是：`<256>e cat in <256>e hat`
  - 新词汇表是

```
  0: ...
  ...
  256: "th"
```

**迭代2**

1. **识别频繁对**  
   - 在文本`<256>e cat in <256>e hat`中，对`<256>e`出现两次

2. **替换和记录**  
   - 用一个尚未使用的新token ID替换`<256>e`，例如`257`。  
   - 新文本是：
     ```
     <257> cat in <257> hat
     ```
   - 更新的词汇表是：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**迭代3**

1. **识别频繁对**  
   - 在文本`<257> cat in <257> hat`中，对`<257> `出现两次（一次在开头，一次在"hat"之前）。

2. **替换和记录**  
   - 用一个尚未使用的新token ID替换`<257> `，例如`258`。  
   - 新文本是：
     ```
     <258>cat in <258>hat
     ```
   - 更新的词汇表是：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- 以此类推

&nbsp;
### 1.4.2 解码部分的具体示例（第1.3节中的步骤3）

- 要恢复原始文本，我们通过按引入顺序的相反顺序将每个token ID替换为其对应的对来逆转过程
- 从最终压缩文本开始：`<258>cat in <258>hat`
-  替换`<258>` → `<257> `：`<257> cat in <257> hat`  
- 替换`<257>` → `<256>e`：`<256>e cat in <256>e hat`
- 替换`<256>` → "th"：`the cat in the hat`

&nbsp;
## 2. 简单的BPE实现

- 下面是上述算法的Python类实现，它模仿了`tiktoken` Python用户界面
- 请注意，上面的编码部分描述了通过`train()`的原始训练步骤；但是，`encode()`方法的工作原理类似（尽管由于特殊token处理看起来更复杂一些）：

1. 将输入文本拆分为单个字节
2. 重复查找并替换（合并）相邻的token（对），当它们匹配学习到的BPE合并中的任何对时（从最高到最低"排名"，即按照它们学习的顺序）
3. 继续合并直到无法应用更多合并
4. 最终的token ID列表就是编码输出

In [4]:
from collections import Counter, deque
from functools import lru_cache
import json


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # For the official OpenAI GPT-2 merges, use a rank dict:
        #  of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with "Ġ"
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including "Ġ" if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pre-trained vocabulary and BPE merges from OpenAI's GPT-2 files.

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json').
            bpe_merges_path (str): Path to the bpe_merges file  (GPT-2 calls it 'vocab.bpe').
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # Convert loaded vocabulary to correct format
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use an existing token ID as a placeholder for '\n'
            # Preferentially use "<|endoftext|>" if available
            fallback_token = next((token for token in ["<|endoftext|>", "Ġ", ""] if token in self.inverse_vocab), None)
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # Load GPT-2 merges and store them with an assigned "rank"
        self.bpe_ranks = {}  # reset ranks
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # If token1 or token2 not in vocab, skip
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(f"Skipping pair {pair} as one token is not in the vocabulary.")

    def encode(self, text, allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
        import re
    
        token_ids = []
    
        # If special token handling is enabled
        if allowed_special is not None and len(allowed_special) > 0:
            # Build regex to match allowed special tokens
            special_pattern = (
                "(" + "|".join(re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)) + ")"
            )
    
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # Encode prefix without special handling
    
                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
    
            text = text[last_index:]  # Remaining part to process normally
    
            # Check if any disallowed special tokens are in the remainder
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")
    
        # If no special tokens, or remaining text after special token split:
        tokens = []
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")
            words = line.split()
            for j, word in enumerate(words):
                if j == 0 and i > 0:
                    tokens.append("Ġ" + word)
                elif j == 0:
                    tokens.append(word)
                else:
                    tokens.append("Ġ" + word)
    
        for token in tokens:
            if token in self.inverse_vocab:
                token_ids.append(self.inverse_vocab[token])
            else:
                token_ids.extend(self.tokenize_with_bpe(token))
    
        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # If we haven't loaded OpenAI's GPT-2 merges, use my approach
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # Uncomment for educational purposes:
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # Skip the next token as it's merged
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # Otherwise, do GPT-2-style merging with the ranks:
        # 1) Convert token_ids back to string "symbols" for each ID
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # Repeatedly merge all occurrences of the lowest-rank pair
        while True:
            # Collect all adjacent pairs
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # Find the pair with the best (lowest) rank
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # If no valid ranked pair is present, we're done
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # Merge all occurrences of that pair
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # If we see (first, second) at position i, merge them
                if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                    new_symbols.append(first + second)  # merged symbol
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # Finally, convert merged symbols back to IDs
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # Add space if not present before a newline
                decoded_string += token
            elif token.startswith("Ġ"):
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Save the vocabulary and BPE merges to JSON files.

        Args:
            vocab_path (str): Path to save the vocabulary.
            bpe_merges_path (str): Path to save the BPE merges.
        """
        # Save vocabulary
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump(self.vocab, file, ensure_ascii=False, indent=2)

        # Save BPE merges as a list of dictionaries
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Load the vocabulary and BPE merges from JSON files.

        Args:
            vocab_path (str): Path to the vocabulary file.
            bpe_merges_path (str): Path to the BPE merges file.
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # Load BPE merges
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge["new_id"]
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

- 上面的`BPETokenizerSimple`类中有大量代码，详细讨论超出了本notebook的范围，但下一节提供了用法的简短概述，以便更好地理解类方法

## 3. BPE实现演示

- 在实践中，我强烈建议使用[tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现专注于可读性和教育目的，而不是性能
- 但是，用法与tiktoken大致相似，只是tiktoken没有训练方法
- 让我们通过查看下面的一些示例来看看我上面的`BPETokenizerSimple` Python代码是如何工作的（详细的代码讨论超出了本notebook的范围）

### 3.1 训练、编码和解码

- 首先，让我们考虑一些示例文本作为我们的训练数据集：

In [5]:
import os
import urllib.request

def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        with urllib.request.urlopen(url) as response, open(target_path, "wb") as out_file:
            out_file.write(response.read())
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")
    return target_path

verdict_path = download_file_if_absent(
    url=(
         "https://raw.githubusercontent.com/rasbt/"
         "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
         "the-verdict.txt"
    ),
    filename="the-verdict.txt",
    search_dirs="."
)

with open(verdict_path, "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

the-verdict.txt already exists in ./the-verdict.txt


- 接下来，让我们初始化并训练BPE分词器，词汇表大小为1,000
- 请注意，由于前面讨论的字节值，词汇表大小默认已经是256，所以我们只是在"学习"744个词汇条目（如果考虑`<|endoftext|>`特殊token和`Ġ`空格token；准确地说，那就是742个）
- 作为比较，GPT-2词汇表是50,257个token，GPT-4词汇表是100,256个token（tiktoken中的`cl100k_base`），GPT-4o使用199,997个token（tiktoken中的`o200k_base`）；与我们上面的简单示例文本相比，它们都有更大的训练集

In [6]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

- You may want to inspect the vocabulary contents (but note it will create a long list)

In [7]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


- This vocabulary is created by merging 742 times (`= 1000 - len(range(0, 256)) - len(special_tokens) - "Ġ" = 1000 - 256 - 1 - 1 = 742`)

In [8]:
print(len(tokenizer.bpe_merges))

742


- This means that the first 256 entries are single-character tokens

- Next, let's use the created merges via the `encode` method to encode some text:

In [9]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [10]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 60, 124, 271, 683, 102, 116, 461, 116, 124, 62]


In [11]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


In [12]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 56
Number of token IDs: 21


- From the lengths above, we can see that a 42-character sentence was encoded into 20 token IDs, effectively cutting the input length roughly in half compared to a character-byte-based encoding

- Note that the vocabulary itself is used in the `decode()` method, which allows us to map the token IDs back into text:

In [13]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257]


In [14]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|>


- Iterating over each token ID can give us a better understanding of how the token IDs are decoded via the vocabulary:

In [15]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .
257 -> <|endoftext|>


- As we can see, most token IDs represent 2-character subwords; that's because the training data text is very short with not that many repetitive words, and because we used a relatively small vocabulary size

- As a summary, calling `decode(encode())` should be able to reproduce arbitrary input texts:

In [16]:
tokenizer.decode(
    tokenizer.encode("This is some text.")
)

'This is some text.'

In [17]:
tokenizer.decode(
    tokenizer.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

### 3.2 Saving and loading the tokenizer

- Next, let's look at how we can save the trained tokenizer for reuse later:

In [18]:
# Save trained tokenizer
tokenizer.save_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [19]:
# Load tokenizer
tokenizer2 = BPETokenizerSimple()
tokenizer2.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

- The loaded tokenizer should be able to produce the same results as before:

In [20]:
print(tokenizer2.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|>


In [21]:
tokenizer2.decode(
    tokenizer2.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

&nbsp;
### 3.3 Loading the original GPT-2 BPE tokenizer from OpenAI

- Finally, let's load OpenAI's GPT-2 tokenizer files

In [22]:
# Download files if not already present in this directory

# Define the directories to search and the files to download
search_directories = [".", "../02_bonus_bytepair-encoder/gpt2_model/"]

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

# Ensure directories exist and download files if needed
paths = {}
for url, filename in files_to_download.items():
    paths[filename] = download_file_if_absent(url, filename, search_directories)

vocab.bpe already exists in ../02_bonus_bytepair-encoder/gpt2_model/vocab.bpe
encoder.json already exists in ../02_bonus_bytepair-encoder/gpt2_model/encoder.json


- Next, we load the files via the `load_vocab_and_merges_from_openai` method:

In [23]:
tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=paths["encoder.json"], bpe_merges_path=paths["vocab.bpe"]
)

- The vocabulary size should be `50257` as we can confirm via the code below:

In [24]:
len(tokenizer_gpt2.vocab)

50257

- We can now use the GPT-2 tokenizer via our `BPETokenizerSimple` object:

In [25]:
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print(token_ids)

[1212, 318, 617, 2420]


In [26]:
print(tokenizer_gpt2.decode(token_ids))

This is some text


- You can double-check that this produces the correct tokens using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library](https://github.com/openai/tiktoken):

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```


&nbsp;
# 4. Conclusion

- That's it! That's how BPE works in a nutshell, complete with a training method for creating new tokenizers or loading the GPT-2 tokenizer vocabular and merges from the original OpenAI GPT-2 model
- I hope you found this brief tutorial useful for educational purposes; if you have any questions, please feel free to open a new Discussion [here](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a)
- For a performance comparison with other tokenizer implementations, please see [this notebook](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb)